In [1]:
import pandas_helper
import lendingclub_helper
import lendingclub_columns
import scikit_helper
import sklearn as sk
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
filename = "finished_train_sample.csv"
df = pandas_helper.readData(filename, lendingclub_columns.get_dtypes_by_name())
df.shape

(1000, 151)

In [3]:
len(set(df["sub_grade"].values))

31

In [4]:
lendingclub_columns.get_feature_names_by_column("sub_grade")
for c in lendingclub_columns.columns:
    if c["name"] == "sub_grade":
        print(c)

[{'name': 'sub_grade', 'dtype': <class 'str'>, 'feature': {'type': 'one-hot encoding', 'values': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5']}}]
[{'name': 'sub_grade', 'dtype': <class 'str'>, 'feature': {'type': 'one-hot encoding', 'values': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5']}}] sooka sub_grade
{'name': 'sub_grade', 'dtype': <class 'str'>, 'feature': {'type': 'one-hot encoding', 'values': ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5', 'F1', 'F2', 'F3', 'F4', 'F5', 'G1', 'G2', 'G3', 'G4', 'G5']}}
{'name': 'sub_grade', 'dtype': <class 'str'>, 'featur